In [4]:
pip install tensorflow keras numpy pandas matplotlib scikit-learn streamlit pillow opencv-python


  Using cached tensorflow-2.20.0-cp311-cp311-win_amd64.whl.metadata (4.6 kB)
Using cached tensorflow-2.20.0-cp311-cp311-win_amd64.whl (331.8 MB)
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
    --------------------------------------- 0.5/39.0 MB 5.6 MB/s eta 0:00:07
   - -------------------------------------- 1.3/39.0 MB 5.6 MB/s eta 0:00:07
   - -------------------------------------- 1.6/39.0 MB 4.2 MB/s eta 0:00:09
   -- ------------------------------------- 2.1/39.0 MB 2.7 MB/s eta 0:00:14
   --- ------------------------------------ 3.4/39.0 MB 3.6 MB/s eta 0:00:10
   ---- ----------------------------------- 4.7/39.0 MB 4.0 MB/s eta 0:00:09
   ------ --------------------------------- 6.0/39.0 MB 4.3 MB/s eta 0:00:08
   ------- -------------------------------- 7.3/39.0 MB 4.7 MB/s eta 0:00:07
   -------- ------------------------------- 8.7/39.0 MB 4.9 MB/s eta 0:00:07
   --------- ------------------------------ 9.7/39.0 MB 5.0 MB/s eta 0:00:06
   ----------- 

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\neeth\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\fault_injection\\fault_injection_service_config_parser.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



In [7]:
# Unified fallback script: TF -> PyTorch -> HOG+RandomForest
import sys, os, random
import numpy as np
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

def try_imports():
    backends = {}
    try:
        import tensorflow as tf
        from tensorflow import keras
        backends['tensorflow'] = (tf, keras)
    except Exception as e:
        backends['tensorflow'] = None
    try:
        import torch
        import torch.nn as nn
        import torch.nn.functional as F
        backends['torch'] = (torch, nn, F)
    except Exception:
        backends['torch'] = None
    return backends

backends = try_imports()

if backends['tensorflow'] is not None:
    # ---------------------------
    # TensorFlow path (preferred)
    # ---------------------------
    tf, keras = backends['tensorflow']
    print("Using TensorFlow", tf.__version__)
    tf.random.set_seed(SEED)

    # Minimal Keras model example (quick smoke test)
    from tensorflow.keras import layers, models
    input_shape = (64, 64, 3)
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Rescaling(1./255),
        layers.Conv2D(16, 3, activation='relu'),
        layers.MaxPool2D(),
        layers.Conv2D(32, 3, activation='relu'),
        layers.GlobalAveragePooling2D(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')   # change classes as needed
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    print("\nTensorFlow path ready. Use this model for training if TF works.\n")
    # Example: `model.fit(...)` with your generators/datasets

elif backends['torch'] is not None:
    # ---------------------------
    # PyTorch path (fallback)
    # ---------------------------
    torch, nn, F = backends['torch']
    print("Tensor available:", torch.__version__)
    torch.manual_seed(SEED)

    class TinyCNN(nn.Module):
        def __init__(self, n_classes=10):
            super().__init__()
            self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
            self.pool = nn.MaxPool2d(2,2)
            self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
            self.gap = nn.AdaptiveAvgPool2d((1,1))
            self.fc = nn.Linear(32, n_classes)
        def forward(self, x):
            x = F.relu(self.conv1(x))
            x = self.pool(x)
            x = F.relu(self.conv2(x))
            x = self.gap(x)
            x = x.view(x.size(0), -1)
            x = self.fc(x)
            return F.log_softmax(x, dim=1)

    model = TinyCNN(n_classes=10)
    print(model)
    print("\nPyTorch path ready. Use this model for training with DataLoader.\n")
    # Example: define optimizer, loss, dataloaders and train

else:
    # ------------------------------------------------------
    # Classical ML fallback: HOG features + RandomForest
    # Works without TF/PyTorch. Good for quick experiments.
    # ------------------------------------------------------
    print("No TensorFlow or PyTorch found — using HOG + RandomForest fallback.")
    # pip install scikit-image scikit-learn opencv-python
    try:
        from skimage.feature import hog
        from skimage.color import rgb2gray
        from sklearn.ensemble import RandomForestClassifier
        from sklearn.model_selection import train_test_split
        from sklearn.metrics import classification_report, confusion_matrix
        import cv2
    except Exception as e:
        print("Missing packages for fallback. Install them with:")
        print("pip install scikit-image scikit-learn opencv-python")
        raise

    # Example loader: expects `dataset_dir` with subfolders per class
    def load_images_hog(dataset_dir, image_size=(128,128), max_per_class=200):
        X, y, labels = [], [], []
        class_names = sorted([d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))])
        for cid, cls in enumerate(class_names):
            cls_dir = os.path.join(dataset_dir, cls)
            files = [os.path.join(cls_dir,f) for f in os.listdir(cls_dir) if f.lower().endswith(('.png','.jpg','.jpeg'))]
            files = files[:max_per_class]
            for fpath in files:
                img = cv2.imread(fpath)
                if img is None:
                    continue
                img = cv2.resize(img, image_size)
                gray = rgb2gray(img[..., ::-1])  # cv2 BGR -> RGB if needed
                feat = hog(gray, pixels_per_cell=(16,16), cells_per_block=(2,2), feature_vector=True)
                X.append(feat)
                y.append(cid)
            labels.append(cls)
        return np.array(X), np.array(y), class_names

    # Example usage (uncomment and set path)
    # dataset_dir = r"C:\path\to\fish_dataset"
    # X, y, class_names = load_images_hog(dataset_dir)
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED, stratify=y)
    # clf = RandomForestClassifier(n_estimators=200, random_state=SEED, n_jobs=-1)
    # clf.fit(X_train, y_train)
    # preds = clf.predict(X_test)
    # print(classification_report(y_test, preds, target_names=class_names))
    print("HOG+RandomForest pipeline defined. Load dataset and run training as shown in comments.")


Tensor available: 2.9.1+cpu
TinyCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (gap): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc): Linear(in_features=32, out_features=10, bias=True)
)

PyTorch path ready. Use this model for training with DataLoader.

